In [3]:
import os
import numpy as np
import openpmd_api as io

# ─── 1) Paramètres ────────────────────────────────────────────────
N        = 100_000         # nombre de particules
P_NORM   = 15.0            # norme moyenne souhaitée (MeV/c)
OUT_DIR  = "3D_Particles_Gauss3D"
OUT_FILE = f"{OUT_DIR}/electrons.bp5"
SPECIES  = "electrons"

# Conversion : 1 MeV/c = 5.3442859e-22 kg·m/s
MEV_C_TO_SI = 5.3442859e-22

# ─── 2) Calcule l'écart-type pour chaque composante ───────────────
# Si px, py, pz ~ N(0, σ²), alors ||p|| suit une distribution Maxwellienne
# E(||p||) = σ * sqrt(8/π) ≈ 1.59577 * σ
sigma_mev = P_NORM / np.sqrt(3)  # approche si chaque composante ~ N(0,σ²)
# Si on veut rigoureux : pour une Maxwell 3D, mean ~ 2σ*sqrt(2/π)
# Donc σ ≈ P_NORM / 1.59577
sigma_mev = P_NORM / 1.59577

print(f"σ composante ~ {sigma_mev:.3f} MeV/c")

# ─── 3) Génère les 3 composantes indépendantes ───────────────────
px_mev = np.random.normal(10, sigma_mev, N)
py_mev = np.random.normal(-10, sigma_mev, N)
pz_mev = np.random.normal(0, sigma_mev, N)

# Conversion en SI
px = px_mev * MEV_C_TO_SI
py = py_mev * MEV_C_TO_SI
pz = pz_mev * MEV_C_TO_SI

# ─── 4) Crée le fichier OpenPMD ───────────────────────────────────
os.makedirs(OUT_DIR, exist_ok=True)
series = io.Series(OUT_FILE, io.Access.create)
series.author            = "auto-generated"
series.openPMD           = "1.1.0"
series.openPMD_extension = 1

it = series.iterations[0]
it.time        = 0.0
it.dt          = 0.0
it.time_unit_SI = 1.0

sp = it.particles[SPECIES]

# Attributs de l'espèce
sp.set_attribute("particleShape", np.uint32(2))
sp.set_attribute("currentDeposition", "Esirkepov")
sp.set_attribute("particlePush",       "Boris")
sp.set_attribute("particleInterpolation", "energyConserving")

# Momentum (kg·m/s)
for comp, data in (("x",px), ("y",py), ("z",pz)):
    rc = sp["momentum"][comp]
    rc.reset_dataset(io.Dataset(data.dtype, [N]))
    rc[:] = data
    rc.unit_SI = 1.0

# Charge, masse et poids
q = np.full(N, -1.602176634e-19)  # C
m = np.full(N,  9.10938356e-31)   # kg
w = np.ones_like(q)               # poids

for name, arr, unit in (
    ("charge", q, 1.0),
    ("mass",   m, 1.0),
    ("weighting", w, 1.0)
):
    rc = sp[name][io.Record_Component.SCALAR]
    rc.reset_dataset(io.Dataset(arr.dtype, [N]))
    rc[:] = arr
    rc.unit_SI = unit

# Sauvegarde
series.flush()
series.close()

print(f"✅ Fichier OpenPMD '{OUT_FILE}' créé avec px,py,pz ~ gaussiennes indépendantes et norme ≈ {P_NORM} MeV/c")


σ composante ~ 9.400 MeV/c
✅ Fichier OpenPMD '3D_Particles_Gauss3D/electrons.bp5' créé avec px,py,pz ~ gaussiennes indépendantes et norme ≈ 15.0 MeV/c
